In [14]:
from random import randint, seed

dimension = 3
perimeter = 100
sensitivity = 3
jsonfilename = "matrices3x3highsensitivity(3).json"

%run generate_matrices_tools.ipynb

In [16]:
generate_new_examples(100000, dimension, perimeter, True, False)

[  71  -76  -88]
[ -38   42   52]
[-100   72  -40] :  B
[ 18561 -10466  -9052]
[-10466   5912   4744]
[ -9052   4744  16784] :  G
(18, 31, 1) :  lincomb_LLL
(19.771, 34.198, 1.0000) :  lincomb_cube
(0, 6, -12) :  sv_LLL
(4.2109, 5.7266, -1.5449) :  sv_cube
(-1.7708, -3.1978, 0.00000) :  lincomb_diff

[ 55  81  75]
[-30 -43 -40]
[ 34 -26  -8] :  B
[15211 -8133  -836]
[-8133  4349   418]
[ -836   418  1896] :  G
(-30, -56, -1) :  lincomb_LLL
(-33.981, -63.452, -1.0000) :  lincomb_cube
(-4, 4, -2) :  sv_LLL
(0.58008, 1.9375, -2.5273) :  sv_cube
(3.9813, 7.4518, 0.00000) :  lincomb_diff



2

# Generating new examples

This notebook's purpose is to generate new examples of *dysfunctional matrices* - matrices for which the cube algorithm gives different result than the LLL algorithm. For now, this code generates only square matrices. It returns a json file containing the *dysfunctional* matrices and their core parameters.

The generating function takes in parameters above.

`dimension`: the shape of the matrix

`perimeter`: defines an interval `[- perimeter, perimeter]` for the integers inside the matrix

`sensitivity`: lol i forgot

`jsonfilename`: name of the ouput file


In [ ]:
# How many matrices are dysfunctional?

iterations = 2000
dim = 100
perimeter = 200
print("dimension: ", dim)
dysfunctional = generate_new_examples(iterations, dim, perimeter, printing=False, functioning=False)
print("There's {} dysfunctional matrices of dimension {} out of total {} instances, percent: {}". format(dysfunctional, dim, iterations, float(100*dysfunctional/iterations)))


dimension:  100


KeyboardInterrupt: 